# Powell & CCD-Accel — Clean Protocol (Golden first, then Backtracking)
**Order of Experiments**  
A. **Powell + Golden** on Ackley (x0 = (-3,-3)) and Branin (x0 = (2,2)) → print Results table + plots  
B. **Powell vs CCD-Accel + Golden** on Rosenbrock (x0 = (-1.5,2.0)) and Ackley (x0 = (4,1)) → print comparison table + plots  
C. **Line-search comparison**: switch to **Backtracking/Armijo** (bracket) and **repeat A & B** (new tables + plots)


In [7]:
# Cell 1 — Imports & Utilities
import numpy as np, math, time, os, zipfile
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Callable, List, Tuple, Dict, Any
import pandas as pd
from caas_jupyter_tools import display_dataframe_to_user

plt.rcParams.update({"figure.dpi": 120})

def norm(x: np.ndarray) -> float:
    return float(np.linalg.norm(x))

def savefig(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches="tight")
    plt.close()

@dataclass
class FWrap:
    f: Callable[[np.ndarray], float]
    name: str
    evals: int = 0
    history_x: List[np.ndarray] = field(default_factory=list)
    history_f: List[float] = field(default_factory=list)
    def __call__(self, x: np.ndarray) -> float:
        fx = float(self.f(x)); self.evals += 1
        self.history_x.append(np.array(x, dtype=float)); self.history_f.append(fx); return fx
    def reset(self):
        self.evals = 0; self.history_x.clear(); self.history_f.clear()

ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [ ]:
# Cell 2 — Test functions and known minima
def ackley(xy: np.ndarray) -> float:
    x, y = xy
    term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))
    term2 = -math.exp(0.5*(math.cos(2*math.pi*x)+math.cos(2*math.pi*y)))
    return term1 + term2 + math.e + 20.0

def branin(xy: np.ndarray) -> float:
    x, y = xy
    a = 1.0; b = 5.1/(4*math.pi**2); c = 5.0/math.pi; r = 6.0; s = 10.0; t = 1.0/(8*math.pi)
    return a*(y - b*x*x + c*x - r)**2 + s*(1 - t)*math.cos(x) + s

def rosenbrock(xy: np.ndarray) -> float:
    x, y = xy
    return 100.0*(y - x*x)**2 + (1 - x)**2

KNOWN_MINIMA = {
    "ackley": [(0.0, 0.0)],
    "branin": [(math.pi, 2.275)],
    "rosenbrock": [(1.0, 1.0)]
}

In [ ]:
# Cell 3 — Line searches
def line_search_golden(f, x, d, a=-3.0, b=3.0, tol=1e-6):
    golden_ratio = (math.sqrt(5)-1)/2
    def phi(alpha): return f(x + alpha*d)
    a1 = a + (1-golden_ratio)*(b-a); a2 = a + golden_ratio*(b-a)
    f1 = phi(a1); f2 = phi(a2)
    while abs(b-a) > tol:
        if f1 < f2: b,a2,f2,a1 = a2,a1,f1, a + (1-golden_ratio)*(b-a); f1 = phi(a1)
        else:       a,a1,f1,a2 = a1,a2,f2, a + golden_ratio*(b-a);    f2 = phi(a2)
    al = 0.5*(a+b); x_new = x + al*d; return x_new, f(x_new), al

def line_search_backtracking(f, x, d, alpha0=0.5, rho=0.7, c=1e-8, max_backtracks=2000):
    # Derivative-free Armijo with finite-diff slope proxy; used only in Section C
    dn = float(np.linalg.norm(d))
    s = d / dn if dn > 1.0 else d.copy()
    fx = f(x); eps = 1e-5
    slope = (f(x + eps*s) - f(x - eps*s)) / (2*eps)
    alpha = alpha0
    for _ in range(max_backtracks):
        x_new = x + alpha * s
        if f(x_new) <= fx + c * alpha * slope:
            return x_new, f(x_new), alpha
        alpha *= rho
    x_new = x + alpha * s
    return x_new, f(x_new), alpha

In [ ]:
# Cell 4 — Optimizers
@dataclass
class OptResult:
    method: str; x0: np.ndarray; x: np.ndarray; f: float; nit_outer: int; fevals: int
    time_sec: float; history_x: List[np.ndarray]; history_f: List[float]

def _run_ls(ls_fn, fwrap, x, d, hist_x, hist_f):
    x, fx, _ = ls_fn(fwrap, x, d)
    hist_x.append(x.copy()); hist_f.append(fx); return x

def powell(fwrap: FWrap, x0: np.ndarray, tol: float = 1e-6, max_outer: int = 1200, ls: str = "golden") -> OptResult:
    n = len(x0); U = [np.eye(n)[i].copy() for i in range(n)]
    x = x0.astype(float).copy(); t0 = time.time()
    hist_x = [x.copy()]; hist_f = [fwrap(x)]
    ls_fn = line_search_golden if ls=="golden" else line_search_backtracking
    k = 0
    while k < max_outer:
        xp = x.copy()
        for i in range(n): x = _run_ls(ls_fn, fwrap, x, U[i], hist_x, hist_f)
        for i in range(n-1): U[i] = U[i+1].copy()
        U[-1] = x - xp
        x = _run_ls(ls_fn, fwrap, x, U[-1], hist_x, hist_f)
        if norm(x - xp) <= tol: break
        k += 1
    T = time.time() - t0
    return OptResult(f"Powell({ls})", x0, x, fwrap(x), k, fwrap.evals, T, hist_x, hist_f)

def ccd_accel(fwrap: FWrap, x0: np.ndarray, tol: float = 1e-6, max_outer: int = 1200, ls: str = "golden") -> OptResult:
    n = len(x0); x = x0.astype(float).copy(); t0 = time.time()
    hist_x = [x.copy()]; hist_f = [fwrap(x)]
    ls_fn = line_search_golden if ls=="golden" else line_search_backtracking
    k = 0
    while k < max_outer:
        xp = x.copy()
        for i in range(n):
            e = np.zeros(n); e[i] = 1.0
            x = _run_ls(ls_fn, fwrap, x, e, hist_x, hist_f)
        d = x - xp
        x = _run_ls(ls_fn, fwrap, x, d, hist_x, hist_f)
        if norm(x - xp) <= tol: break
        k += 1
    T = time.time() - t0
    return OptResult(f"CCD-Accel({ls})", x0, x, fwrap(x), k, fwrap.evals, T, hist_x, hist_f)

In [ ]:
# Cell 5 — Plotters
def plot_trajectory(fun, res: OptResult, fname: str, minima, grid_N: int = 300):
    xs = np.array([p[0] for p in res.history_x]); ys = np.array([p[1] for p in res.history_x])
    Xs = [res.x0[0], res.x[0]] + [m[0] for m in minima]; Ys = [res.x0[1], res.x[1]] + [m[1] for m in minima]
    xmin, xmax = min(Xs)-2, max(Xs)+2; ymin, ymax = min(Ys)-2, max(Ys)+2
    X = np.linspace(xmin, xmax, grid_N); Y = np.linspace(ymin, ymax, grid_N)
    XX, YY = np.meshgrid(X, Y); ZZ = np.zeros_like(XX)
    for i in range(grid_N):
        for j in range(grid_N):
            ZZ[i, j] = fun(np.array([XX[i, j], YY[i, j]]))
    plt.figure(); plt.contour(XX, YY, ZZ, levels=30)
    plt.plot(xs, ys, marker='o', linewidth=1)
    plt.scatter([res.x0[0]], [res.x0[1]], marker='s', s=40, label='start')
    plt.scatter([res.x[0]], [res.x[1]], marker='*', s=80, label='final')
    for (mx, my) in minima: plt.scatter([mx], [my], marker='x', s=50, label='global min')
    plt.legend(); plt.title(f"Trajectory: {res.method}"); plt.xlabel("x"); plt.ylabel("y")
    savefig(fname)

def plot_convergence(res: OptResult, fname: str):
    plt.figure(); plt.plot(res.history_f, marker='o', linewidth=1)
    plt.xlabel("iteration (line-search steps)"); plt.ylabel("f(x)")
    plt.title(f"Convergence: {res.method}"); savefig(fname)

In [ ]:
# Cell 6 — A) Powell + Golden on Ackley/Branin
tol=1e-6; max_outer=600
rows = []
for (name, ffun, x0) in [('ackley', ackley, np.array([-3.,-3.])), ('branin', branin, np.array([2.,2.]))]:
    fw = FWrap(ffun, name)
    res = powell(fw, x0, tol=tol, max_outer=max_outer, ls='golden')
    traj = f"/mnt/data/plot {name} trajectory_golden.png"
    conv = f"/mnt/data/plot {name} convergence_golden.png"
    plot_trajectory(ffun, res, traj, KNOWN_MINIMA[name])
    plot_convergence(res, conv)
    rows.append({'function':name,'start':tuple(x0.tolist()),'method':'Powell','x found':tuple(res.x.tolist()),
                 'f(x found)':res.f,'iterations':res.nit_outer,'time (s)':res.time_sec,
                 'trajectory_png':traj,'convergence_png':conv})
df_A = pd.DataFrame(rows)
display_dataframe_to_user("A) Powell + Golden on Ackley/Branin", df_A)
df_A

NameError: name 'display_dataframe_to_user' is not defined

In [ ]:
# Cell 7 — B) Powell vs CCD-Accel + Golden on Rosenbrock/Ackley
tol=1e-6; max_outer=800
rows = []
for (name, ffun, x0) in [('rosenbrock', rosenbrock, np.array([-1.5, 2.0])),
                         ('ackley', ackley, np.array([4.0, 1.0]))]:
    # Powell
    fw1 = FWrap(ffun, name); res1 = powell(fw1, x0, tol=tol, max_outer=max_outer, ls='golden')
    traj1 = f"/mnt/data/plot {name} trajectory_powell_golden.png"
    conv1 = f"/mnt/data/plot {name} convergence_powell_golden.png"
    plot_trajectory(ffun, res1, traj1, KNOWN_MINIMA[name]); plot_convergence(res1, conv1)
    rows.append({'function':name,'start':tuple(x0.tolist()),'method':'Powell','x found':tuple(res1.x.tolist()),
                 'f(x found)':res1.f,'iterations':res1.nit_outer,'time (s)':res1.time_sec,
                 'trajectory_png':traj1,'convergence_png':conv1})
    # CCD-Accel
    fw2 = FWrap(ffun, name); res2 = ccd_accel(fw2, x0, tol=tol, max_outer=max_outer, ls='golden')
    traj2 = f"/mnt/data/plot {name} trajectory_ccd_golden.png"
    conv2 = f"/mnt/data/plot {name} convergence_ccd_golden.png"
    plot_trajectory(ffun, res2, traj2, KNOWN_MINIMA[name]); plot_convergence(res2, conv2)
    rows.append({'function':name,'start':tuple(x0.tolist()),'method':'CCD-Accel','x found':tuple(res2.x.tolist()),
                 'f(x found)':res2.f,'iterations':res2.nit_outer,'time (s)':res2.time_sec,
                 'trajectory_png':traj2,'convergence_png':conv2})
df_B = pd.DataFrame(rows)
display_dataframe_to_user("B) Powell vs CCD-Accel + Golden on Rosenbrock/Ackley", df_B)
df_B

In [ ]:
# Cell 8 — C) Line-search comparison: switch to Backtracking (Armijo) and repeat A & B
tol=1e-6; max_outer_A=600; max_outer_B=800
# C1) Powell on Ackley/Branin with backtracking
rows_C1 = []
for (name, ffun, x0) in [('ackley', ackley, np.array([-3.,-3.])), ('branin', branin, np.array([2.,2.]))]:
    fw = FWrap(ffun, name)
    res = powell(fw, x0, tol=tol, max_outer=max_outer_A, ls='backtracking')
    traj = f"/mnt/data/plot {name} trajectory_backtracking.png"
    conv = f"/mnt/data/plot {name} convergence_backtracking.png"
    plot_trajectory(ffun, res, traj, KNOWN_MINIMA[name]); plot_convergence(res, conv)
    rows_C1.append({'function':name,'start':tuple(x0.tolist()),'method':'Powell','x found':tuple(res.x.tolist()),
                    'f(x found)':res.f,'iterations':res.nit_outer,'time (s)':res.time_sec,
                    'trajectory_png':traj,'convergence_png':conv})
df_C1 = pd.DataFrame(rows_C1)
display_dataframe_to_user("C1) Powell + Backtracking on Ackley/Branin", df_C1)

# C2) Powell vs CCD-Accel on Rosenbrock/Ackley with backtracking
rows_C2 = []
for (name, ffun, x0) in [('rosenbrock', rosenbrock, np.array([-1.5, 2.0])),
                         ('ackley', ackley, np.array([4.0, 1.0]))]:
    fw1 = FWrap(ffun, name); res1 = powell(fw1, x0, tol=tol, max_outer=max_outer_B, ls='backtracking')
    traj1 = f"/mnt/data/plot {name} trajectory_powell_backtracking.png"
    conv1 = f"/mnt/data/plot {name} convergence_powell_backtracking.png"
    plot_trajectory(ffun, res1, traj1, KNOWN_MINIMA[name]); plot_convergence(res1, conv1)
    rows_C2.append({'function':name,'start':tuple(x0.tolist()),'method':'Powell','x found':tuple(res1.x.tolist()),
                    'f(x found)':res1.f,'iterations':res1.nit_outer,'time (s)':res1.time_sec,
                    'trajectory_png':traj1,'convergence_png':conv1})
    fw2 = FWrap(ffun, name); res2 = ccd_accel(fw2, x0, tol=tol, max_outer=max_outer_B, ls='backtracking')
    traj2 = f"/mnt/data/plot {name} trajectory_ccd_backtracking.png"
    conv2 = f"/mnt/data/plot {name} convergence_ccd_backtracking.png"
    plot_trajectory(ffun, res2, traj2, KNOWN_MINIMA[name]); plot_convergence(res2, conv2)
    rows_C2.append({'function':name,'start':tuple(x0.tolist()),'method':'CCD-Accel','x found':tuple(res2.x.tolist()),
                    'f(x found)':res2.f,'iterations':res2.nit_outer,'time (s)':res2.time_sec,
                    'trajectory_png':traj2,'convergence_png':conv2})
df_C2 = pd.DataFrame(rows_C2)
display_dataframe_to_user("C2) Powell vs CCD-Accel + Backtracking on Rosenbrock/Ackley", df_C2)

# Show compact heads
df_C1, df_C2.head()